In [1]:
import numpy as np
import pandas as pd

import statsmodels.api as sm

import time
import datetime

import scipy.stats as sts

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression

C:\Users\Denis\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
data = pd.read_json('table.jsonl', lines=True)

Gold = a * time + b

In [3]:
Y, X = data[data['type']=='interval'].dropna(subset=[['gold','time']])['gold'].as_matrix().reshape(-1, 1), data[data['type']=='interval'].dropna(subset=[['gold','time']])['time'].as_matrix().reshape(-1, 1)

In [4]:
def smodt(X,Y):
    X = sm.add_constant(X)
    mod = sm.OLS(Y, X)
    res = mod.fit()
    print('\n', res.summary())
    
def Testing_grid_reg(X_train, Y_train):
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    
    grid = {'fit_intercept':[True,False], 
            'normalize':[True,False], 
            'copy_X':[True, False]}
    
    clf = LinearRegression(n_jobs = 5)
    
    start_time = datetime.datetime.now()
    gs = GridSearchCV(clf, scoring='r2', param_grid=grid, cv=cv, return_train_score=True)
    gs.fit(X_train,Y_train)
    
    print ('Time elapsed:', datetime.datetime.now() - start_time)
    
    means = gs.cv_results_['mean_test_score']
    stds = gs.cv_results_['std_test_score']
    
    for mean, std, params in zip(means, stds, gs.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    lm = gs.best_estimator_
    
    lm.fit(X,Y)
    params = np.append(lm.intercept_,lm.coef_)
    predictions = lm.predict(X)
    
    newX = pd.DataFrame({"Constant":np.ones(len(X))}).join(pd.DataFrame(X))
    MSE = (sum((X-predictions)**2))/(len(newX)-len(newX.columns))

    # Note if you don't want to use a DataFrame replace the two lines above with
    # newX = np.append(np.ones((len(X),1)), X, axis=1)
    # MSE = (sum((y-predictions)**2))/(len(newX)-len(newX[0]))

    var_b = MSE*(np.linalg.inv(np.dot(newX.T,newX)).diagonal())
    sd_b = np.sqrt(var_b)
    ts_b = params/ sd_b

    p_values =[2*(1-sts.t.cdf(np.abs(i),(len(newX)-1))) for i in ts_b]
    
    sd_b = np.round(sd_b,3)
    ts_b = np.round(ts_b,3)
    p_values = np.round(p_values,3)
    params = np.round(params,4)
    
    myDF3 = pd.DataFrame()
    myDF3["Coefficients"],myDF3["Standard Errors"],myDF3["t values"],myDF3["Probabilites"] = [params,sd_b,ts_b,p_values]
    
    print ('\n', myDF3)
    
    smodt(X,Y)
    return gs.best_estimator_ 

Значимы​​ ли​​ коэффициенты​​ регрессии?​​ Нужно​​ ли​​ использовать​​ intercept​​ в
данной​​ модели?​​ Какой​​ смысл​​ в​​ терминах​​ игры​​ имеет​​ коэффициент​​ наклона?

In [5]:
clf = Testing_grid_reg(X,Y)

Time elapsed: 0:00:02.266487
0.716 (+/-0.005) for {'copy_X': True, 'fit_intercept': True, 'normalize': True}
0.716 (+/-0.005) for {'copy_X': True, 'fit_intercept': True, 'normalize': False}
0.636 (+/-0.004) for {'copy_X': True, 'fit_intercept': False, 'normalize': True}
0.636 (+/-0.004) for {'copy_X': True, 'fit_intercept': False, 'normalize': False}
0.716 (+/-0.005) for {'copy_X': False, 'fit_intercept': True, 'normalize': True}
0.716 (+/-0.005) for {'copy_X': False, 'fit_intercept': True, 'normalize': False}
0.636 (+/-0.004) for {'copy_X': False, 'fit_intercept': False, 'normalize': True}
0.636 (+/-0.004) for {'copy_X': False, 'fit_intercept': False, 'normalize': False}

    Coefficients  Standard Errors  t values  Probabilites
0    -3867.9249           50.486   -76.614           0.0
1        5.8809            0.026   229.697           0.0

                             OLS Regression Results                            
Dep. Variable:                      y   R-squared:               

Оба коэффициента значимы - $|t_{value}| > P_{value} $. 

В начале игры(time = 0), золото(intercept) будет равно -3867.9249, что не отражает реальность, а также говорит о наличии других проблем (например, мультикорреляция). 

Коэффициент наклона говорит нам, что увеличение золота на одну единицу приводит к увеличению времени на 5.8809 сек, или каждые 5.8809 секунд приносят игроку 1 голд.

In [6]:
heroes = pd.read_json('heroes.json')
heroes.head()

,id,localized_name,name
0,1,Anti-Mage,antimage
1,2,Axe,axe
2,3,Bane,bane
3,4,Bloodseeker,bloodseeker
4,5,Crystal Maiden,crystal_maiden


In [7]:
def name(id):
    if id in heroes['id'].values:        
        name = (heroes['name'][heroes['id'] == id].tolist())[0]
        return 'npc_dota_hero_'+name
    else:
        return np.nan
name(34)

'npc_dota_hero_tinker'

In [8]:
data[(data['type']=='DOTA_COMBATLOG_DEATH')].isnull().sum()[lambda x: x!= 3430]

attackerhero        0
attackerillusion    0
attackername        0
attackerteam        1
inflictor           0
location            0
sourcename          0
targethero          0
targetillusion      0
targetname          0
targetsourcename    0
targetteam          0
tick                0
time                0
type                0
value               0
dtype: int64

'attackername', 'targetname', 'time', 'location'

In [9]:
Combat = data[(data['type']=='DOTA_COMBATLOG_DEATH')][['attackername', 'targetname', 'time', 'location']].sort_values(by=['time'])

assists, kills, time, hero_id

In [10]:
def ak(s):
    rez = pd.DataFrame(columns = ['assists', 'kills', 'time','hero_id'])
    for t in data[data[s]>0][s].dropna().unique():
        for h in data[data[s]>0]['hero_id'].dropna().unique():
            tmp = data[(data[s] == t)& (data['hero_id'] == h)][['assists', 'kills', 'time','hero_id']].dropna().sort_values(by=['time']).head(1)
            if len(tmp) == 1:
                rez = rez.append(tmp)
    rez['hero_name'] = rez['hero_id'].apply(lambda x: name(x))
    return rez.reset_index()[['hero_name','time',s]].sort_values(by=['time'])

In [11]:
kill = ak('kills')
assist = ak('assists')

In [12]:
def Participants(tes, tes2):
    rez = pd.DataFrame()
    
    for row in tes.as_matrix():
        tmp = Combat[(Combat['attackername']==row[0]) & 
                     (Combat['targetname'].apply(lambda x: True if x.split('_')[2] in ['hero'] else False)) & 
                     (Combat['time']<row[1])].as_matrix()
        tmp2 = tes2[tes2['time']==row[1]]['hero_name'].as_matrix()
        if len(tmp)>1:
            if tmp[0][2]!=tmp[len(tmp)-1][2]:
                rez = pd.concat([rez, (pd.DataFrame([[tmp[len(tmp)-1][1], tmp[len(tmp)-1][2], row[1], np.unique(np.append(tmp2, tmp[len(tmp)-1][0]))]], 
                                                    columns = ['targethero', 'start_time', 'end_time','kill_participants']))], 
                                axis=0)
            else:
                rez = pd.concat([rez, (pd.DataFrame([[tmp[0][1], tmp[0][2], row[1], np.unique(np.append(tmp2,tmp[0][0]))]], 
                                                    columns = ['targethero', 'start_time', 'end_time','kill_participants']))], 
                                axis=0)
        else:
            rez = pd.concat([rez, (pd.DataFrame([[tmp[0][1], tmp[0][2], row[1], np.unique(np.append(tmp2,tmp[0][0]))]], 
                                                columns = ['targethero', 'start_time', 'end_time','kill_participants']))], 
                            axis=0)
    return rez.reset_index()[['targethero', 'start_time', 'end_time','kill_participants']]

In [13]:
rez = Participants(kill, assist)

In [14]:
with open('kills.json', 'w') as f:
    f.write(rez.to_json(orient='records'))